In [10]:
import json
from pathlib import Path
import pandas as pd
import geopandas as gpd

In [2]:
ROOT = Path('../..')

OUTPUT_DIR = ROOT / 'src/_data/areas/'

REF_DIR = OUTPUT_DIR / 'reference'
REF_DIR.mkdir(parents=True, exist_ok=True)

MAPS_DIR = OUTPUT_DIR / 'maps'
MAPS_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / 'src/generated/areas/constituency/_data/'
DATA_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
geo = gpd.read_file(ROOT / "working/upstream/uk-constituencies-2024.geojson").query('PCON24CD.str.startswith("E")')
geo.crs = None

In [24]:
mapping = pd.read_csv(ROOT / "working/upstream/uk-constituencies-2024-changes.csv").query('PCON24CD.notna()').set_index('PCON24CD')[['PCON21CD']]

In [27]:
geo.loc[:, ['PCON24CD', 'PCON24NM']].set_index('PCON24CD').join(mapping).reset_index().to_json(REF_DIR / 'pcon.json', orient='records')

In [17]:
geo.to_file(MAPS_DIR / 'pcon.geojson', coordinate_precision=5)

In [7]:
with open(MAPS_DIR / 'pcon.geojson') as geojson:
    d = json.load(geojson) 

In [9]:
res = {}
for f in d['features']:
    res[f['properties']['PCON24CD']] = {
      "type": "FeatureCollection",
      "features": [f]
    }

with open(DATA_DIR / 'map.json', 'w') as f:
    json.dump(res, f)